## Preprocessing

In [31]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [32]:
#Install Keras-tuner
!pip install keras-tuner

In [33]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN","NAME"], axis = 1, inplace = True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [34]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [35]:
# Look at APPLICATION_TYPE value counts for binning
counts_AT = application_df.APPLICATION_TYPE.value_counts()
counts_AT

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [36]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(counts_AT[counts_AT<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [37]:
# Look at CLASSIFICATION value counts for binning
counts_class = application_df.CLASSIFICATION.value_counts()
counts_class

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [38]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df.CLASSIFICATION.value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [39]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(counts_class[counts_class<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [40]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [41]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns='IS_SUCCESSFUL')


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [42]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

Original Model

In [43]:
units_1 = 8
units_2 = 5
input_features = len(X_train_scaled[0])
nn_model1 = tf.keras.models.Sequential()

In [44]:
# First hidden layer
nn_model1.add(tf.keras.layers.Dense(units=units_1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn_model1.add(tf.keras.layers.Dense(units=units_2, activation = "relu"))

# Output layer
nn_model1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 8)                 352       
                                                                 
 dense_7 (Dense)             (None, 5)                 45        
                                                                 
 dense_8 (Dense)             (None, 1)                 6         
                                                                 
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [45]:
# Compile the model
nn_model1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model (epochs = 100)
fit_model = nn_model1.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6087 - accuracy: 0.6946
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5676 - accuracy: 0.7262
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5602 - accuracy: 0.7274
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5574 - accuracy: 0.7282
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5556 - accuracy: 0.7298
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5545 - accuracy: 0.7291
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7294
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.7296
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5520 - accuracy: 0.7294
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5515 - accura

In [46]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - loss: 0.5539 - accuracy: 0.7298 - 417ms/epoch - 2ms/step
Loss: 0.5538860559463501, Accuracy: 0.7297959327697754


Optimisation 2

In [47]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Will be using all the input features (43).
units_1 = 8
units_2 = 8
units_3 = 8
input_features = len(X_train_scaled[0])
nn_model2 = tf.keras.models.Sequential()

In [48]:
# First hidden layer
nn_model2.add(tf.keras.layers.Dense(units=units_1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn_model2.add(tf.keras.layers.Dense(units=units_2, activation = "relu"))

# Third hidden layer
nn_model2.add(tf.keras.layers.Dense(units=units_3, activation = "relu"))

# Output layer
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 8)                 352       
                                                                 
 dense_10 (Dense)            (None, 8)                 72        
                                                                 
 dense_11 (Dense)            (None, 8)                 72        
                                                                 
 dense_12 (Dense)            (None, 1)                 9         
                                                                 
Total params: 505
Trainable params: 505
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Compile the model
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model (epochs = 200)
fit_model = nn_model2.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 1ms/step - loss: 0.5991 - accuracy: 0.6921
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5648 - accuracy: 0.7251
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5595 - accuracy: 0.7277
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5560 - accuracy: 0.7297
Epoch 5/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5545 - accuracy: 0.7289
Epoch 6/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7303
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5525 - accuracy: 0.7302
Epoch 8/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7310
Epoch 9/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5512 - accuracy: 0.7316
Epoch 10/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5508 - accura

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5541 - accuracy: 0.7277 - 289ms/epoch - 1ms/step
Loss: 0.5540746450424194, Accuracy: 0.7276967763900757


Optimisation 3

In [51]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Will be using all the input features (43).
units_1 = 50
units_2 = 30
input_features = len(X_train_scaled[0])
nn_model3 = tf.keras.models.Sequential()

In [52]:
# First hidden layer
nn_model3.add(tf.keras.layers.Dense(units=units_1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn_model3.add(tf.keras.layers.Dense(units=units_2, activation = "tanh"))

# Output layer
nn_model3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 50)                2200      
                                                                 
 dense_14 (Dense)            (None, 30)                1530      
                                                                 
 dense_15 (Dense)            (None, 1)                 31        
                                                                 
Total params: 3,761
Trainable params: 3,761
Non-trainable params: 0
_________________________________________________________________


In [53]:
# Compile the model
nn_model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model (epoch = 100)
fit_model = nn_model3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5710 - accuracy: 0.7178
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5566 - accuracy: 0.7273
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5529 - accuracy: 0.7298
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5510 - accuracy: 0.7290
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5499 - accuracy: 0.7317
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7295
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7304
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5471 - accuracy: 0.7318
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7326
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accura

In [54]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5572 - accuracy: 0.7279 - 294ms/epoch - 1ms/step
Loss: 0.5572364926338196, Accuracy: 0.7279300093650818


Optimisation 4 Hyperband Tuning

In [60]:
def create_model(hp):
    nn_model4 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model4.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model4.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model4.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model4

In [61]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [63]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

In [64]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 5,
 'num_layers': 6,
 'units_0': 5,
 'units_1': 7,
 'units_2': 7,
 'units_3': 3,
 'units_4': 9,
 'units_5': 7,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0012'}

In [59]:

# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5770 - accuracy: 0.7333 - 402ms/epoch - 2ms/step
Loss: 0.5769785642623901, Accuracy: 0.7332944869995117
268/268 - 0s - loss: 0.5767 - accuracy: 0.7332 - 403ms/epoch - 2ms/step
Loss: 0.5766885280609131, Accuracy: 0.7331778407096863
268/268 - 0s - loss: 0.5734 - accuracy: 0.7332 - 342ms/epoch - 1ms/step
Loss: 0.5734347701072693, Accuracy: 0.7331778407096863
